Das GloVe Embedding basiert auf nicht Normalisiertem Text. Deshalb muss das GloVe Embedding naürlich ebenfalls, wie die Reviews normalisiert werden, um dies verwenden zu können. Sollte sich durch die Lemmatisierung oder die Stammformbildung das selbe Token aus zwei Wörtern ergeben, wird der Mittelwert der Vektoren als neuer Wert angenommen.

In [1]:
import os
import numpy as np

In [2]:
def load_embedding(file_path):
    with open (file_path,'r', encoding ="utf-8") as f:
        lines = f.readlines()
        embedding = dict()
        for line in lines:
            parts = line.split()
            embedding[parts[0]] = np.asarray(parts[1:], dtype='float32')
        return embedding

In [3]:
file_path = os.path.join("data", "models", "glove.6B.100d.txt")
glove_embedding = load_embedding(file_path)

In [4]:
from own.processing_classes import LemmaStemmer
lemma_stemmer = LemmaStemmer()

In [5]:
word_list = glove_embedding.keys()
word_list_stemmed = lemma_stemmer.perform(word_list)

In [6]:
final_glove_dict = dict()
vocab_dict = dict()
file_path = os.path.join("data", "models", "edited_glove.txt")


for old, new in zip(word_list, word_list_stemmed):
    # check if lemmastemmed word is already in "vocab". If -> Mean Else -> Append it
    if new in vocab_dict.keys(): 
        vocab_dict[new] = vocab_dict[new] + 1
        count_mean = vocab_dict[new]
        final_glove_dict[new] = np.array((count_mean-1)/count_mean*final_glove_dict[new] + 1/count_mean*glove_embedding.get(old))
    else:
        vocab_dict[new] = 1
        final_glove_dict[new] = glove_embedding.get(old)

In [3]:
file_path = os.path.join("data", "models", "glove.6B.100d.txt")
glove_embedding = load_embedding(file_path)

In [17]:
%%time
from datetime import datetime

data = ""
i= 1
for k, v in final_glove_dict.items():
    print("Converting item: {} of {} ".format(i, len(final_glove_dict)), end = "\r")
    if i == 1:
        data = k +" "+ " ".join(v.astype(str))
    data = data + "\n" + k +" "+ " ".join(v.astype(str))
    i = i + 1

Wall time: 19h 11min 36sof 343508 


In [18]:
file_path = os.path.join("data", "models", "own.glove.6B.100d.txt")
with open(file_path, "w", encoding="utf-8") as f:
    f.write(data)
    #np.savetxt(f, list(final_glove_dict.items())[0][1])

In [19]:
file_path = os.path.join("data", "models", "own.glove.6B.100d.txt")
own_glove_embedding = load_embedding(file_path)

In [20]:
len(own_glove_embedding)

343508